In [1]:
# python libraries
import os
import sys
from pathlib import Path
from operator import methodcaller
from collections import OrderedDict
import dataclasses
from dataclasses import dataclass
from typing import (
    List,
    Tuple,
    Dict,
    Any,
    Mapping
)
from enum import Enum
# adding the path
if not str(Path(os.getcwd()).parent) in sys.path:
    sys.path.append(str(Path(os.getcwd()).parent))

# numpy
import numpy as np

# torch
import torch
import torchvision
from torch import nn
from torchvision import (
    transforms,
    datasets
    )
from torch.utils.data import DataLoader

try:
    from torchmetrics import Accuracy
except:
    print(f"[INFO] Installing the torchmetrics")
    %pip install torchmetrics
    from torchmetrics import Accuracy

# helper function
try:
    import my_helper as helper
except:
    print("[INFO] Downloading the helper function from github")
    import requests
    response = requests.get("https://raw.githubusercontent.com/Lashi0812/PyTorch2/master/my_helper.py")
    with open("my_helper.py" ,"wb") as f:
        f.write(response.content)
    import my_helper as helper

# VGG Network

In [16]:
layer_arch = Tuple[int,int]

@dataclass(eq=False)
class VGG(helper.Classifier):
    arch:Tuple[layer_arch]
    lr:int = 0.01
    num_classes:int =10

    def  __post_init__(self):
        super().__init__()
        conv_blocks = []
        for num_convs,out_channels in self.arch:
            conv_blocks.append(self.vgg_block(num_convs,out_channels))
        self.net = nn.Sequential(
            *conv_blocks,nn.Flatten(),
            nn.LazyLinear(4096),nn.ReLU(),nn.Dropout(0.5),
            nn.LazyLinear(4096),nn.ReLU(),nn.Dropout(0.5),
            nn.LazyLinear(self.num_classes)
        )
        

    @staticmethod
    def vgg_block(num_convs,out_channels):
        layers = []
        for _ in range(num_convs):
            layers.append(nn.LazyConv2d(out_channels,kernel_size=3,padding=1))
            layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
        return nn.Sequential(*layers)

    
    @staticmethod
    def init_vgg(module):
        if isinstance(module,nn.Linear|nn.Conv2d):
            nn.init.kaiming_normal_(module.weight)


In [17]:
arch = ((1,64),(1,128),(2,256),(2,512),(2,512))
VGG(arch).layer_summary((1,1,224,224))

Sequential      output shape :(1, 64, 112, 112)
Sequential      output shape :(1, 128, 56, 56)
Sequential      output shape :(1, 256, 28, 28)
Sequential      output shape :(1, 512, 14, 14)
Sequential      output shape :(1, 512, 7, 7)
Flatten         output shape :(1, 25088)
Linear          output shape :(1, 4096)
ReLU            output shape :(1, 4096)
Dropout         output shape :(1, 4096)
Linear          output shape :(1, 4096)
ReLU            output shape :(1, 4096)
Dropout         output shape :(1, 4096)
Linear          output shape :(1, 10)


In [25]:
# Training process is not same as original paper

#? they start 1st conv block with 64 here we start with 16
arch = ((1,16),(1,32),(2,64),(2,128),(2,128))

#? we use the same learning rate as in the original paper , 
#? we didn't use the momentum and weight decay
model = VGG(arch,lr=0.01)

#? data we use is Fashion mnist
#? batch we reduce 128 original paper use the 256
#? there is not transform such horizontal flip
#? we didn't use the different scale for training image.
data = helper.FashionMNIST(batch_size=128,resize=(224,224))

#? training done with 10 epoch original they trained for 74 epoch
trainer = helper.Trainer(max_epochs=1)

#? Initialization is totally different 
#? original paper use the different initialization for mostly pertained from the shallow model
model.apply_init([next(iter(data.train_dataloader()))[0]],model.init_vgg)

trainer.fit(model,data)

c:\Users\Lenovo\anaconda3\envs\py310\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Training the model in cpu


  0%|          | 0/1 [00:00<?, ?it/s]

NameError: name 'model' is not defined